# get pthick and pfull for 14-18km

In [2]:
import xarray as xr
import dask
import numpy as np
import analysis_parameters as ap
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [3]:
region="GT_14-18km"
chunk_dict={"time":240//3, "lat":200,"lon":200}

## SCREAM

In [3]:
ds = xr.open_dataset(ap.get_file("SCREAMr0.25deg", "GT_14-18km", "cli"), chunks=chunk_dict).cli
ds

,Array,Chunk
Bytes,5.87 GiB,46.39 MiB
Shape,"(240, 240, 19, 1440)","(80, 80, 19, 100)"
Count,136 Tasks,135 Chunks
Type,float32,numpy.ndarray


In [4]:
p = xr.open_dataset(ap.get_file("SCREAMr0.25deg", "GT_14-18km", "cli"), chunks=chunk_dict).cli.lev
print(p.shape)
dp = xr.DataArray(np.zeros(p.shape),dims=p.dims, coords=p.coords, attrs=p.attrs)
dp[1:] = abs(p[:-1].values-p[1:].values)
dp[0] = dp[1].values
print(p[0:3].values, p.shape, dp[0:3].values, dp.shape)

(19,)
[74.99975967 78.92103577 82.59738922] (19,) [3.92127609 3.92127609 3.67635345] (19,)


In [5]:
dp3D = dp.expand_dims({"time":ds.time, "lat":ds.lat, "lon":ds.lon}).astype("float32")
xr.Dataset({"pthick":dp3D}).to_netcdf("/work/bb1153/b380883/GT/GT_14-18km_SCREAMr0.25deg_pthick_20200130-20200228.nc")

del dp3D

In [6]:
p3D = p.expand_dims({"time":ds.time, "lat":ds.lat, "lon":ds.lon}).astype("float32")
xr.Dataset({"pfull":p3D}).to_netcdf("/work/bb1153/b380883/GT/GT_14-18km_SCREAMr0.25deg_pfull_20200130-20200228.nc")

del p3D

## NICAM

In [7]:
chunk_dict={"time":240//6, "lat":200,"lon":200}
p = xr.open_dataset(ap.get_file("NICAM", region, "pfull"), chunks=chunk_dict).pfull
p

,Array,Chunk
Bytes,171.81 GiB,67.14 MiB
Shape,"(240, 11, 1706, 10240)","(40, 11, 200, 200)"
Count,2809 Tasks,2808 Chunks
Type,float32,numpy.ndarray


In [ ]:
dp = xr.DataArray(np.zeros(p.shape),dims=p.dims, coords=p.coords, attrs=p.attrs)
dp[1:] = abs(p[:-1].values-p[1:])
dp[0] = dp[1].values
print(p[0:3].values, p.shape, dp[0:3].values, dp.shape)

[                                        ] | 1% Completed | 38min  6.8s
